In [7]:
pip install ray ConfigSpace

  Using cached ray-2.39.0-cp310-cp310-manylinux2014_x86_64.whl.metadata (17 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 134.0 MB/s eta 0:00:0000:0100:01
  Created wheel for ConfigSpace: filename=ConfigSpace-1.2.0-py3-none-any.whl size=115855 sha256=7142c397a045e2d854e039b56dcab96fc9c303b7e654e45f85f15227e25d4acb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5f/1f/4b/450c9c9d1ede888e354984d9eba7e60f13878fa079d502d52e
Successfully built ConfigSpace
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install bayesian-optimization

In [48]:
import bayesopt_search2 as b2

In [46]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import ray
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler

In [93]:
reload(b2)

<module 'bayesopt_search2' from '/home/ec2-user/SageMaker/projects/bayesopt_search2.py'>

In [94]:
"""This example demonstrates the usage of BayesOpt with Ray Tune.

It also checks that it is usable with a separate scheduler.

Requires the BayesOpt library to be installed (`pip install bayesian-optimization`).
"""
import time

from ray import train, tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch


def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        train.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing"
    )
    args, _ = parser.parse_known_args()

#    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0}) # utility function is broken
    # need to manage numerical instability
    algo = b2.BayesOptSearch(bo_args={'batch_size': 50, 'keys': ['width', 'height'], 'n_procs': 4,
                                      'hp_space': {'width':[.1, 2], 'height':[.1, .9]}, 
                                      'hp_types': {'height': 'float', 'width': 'float'}})
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    tuner = tune.Tuner(
        #easy_objective,
        tune.with_resources(easy_objective, resources={"CPU": 24}),
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            search_alg=algo,
            scheduler=scheduler,
            num_samples=10 if args.smoke_test else 20,
        ),
        run_config=train.RunConfig(
            name="my_exp",
        ),
        param_space={
            "steps": 100,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
        },
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

2024-11-15 19:20:20,551	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/ec2-user/ray_results/my_exp' in 0.0145s.
2024-11-15 19:20:20,559	INFO tune.py:1041 -- Total run time: 155.47 seconds (155.43 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 100, 'width': 19.148309730364033, 'height': -95.96081184529413}


In [95]:
algo.searcher.test_optimizer.

{'batch_size': 50,
 'historical_parameters': ['.5,.5',
  '.4,.4',
  '11.973169683940732,46.39878836228101',
  '19.01428612819832,-25.091976230527507',
  '3.119890406724053,-68.79627191151269',
  '14.161451555920909,20.223002348641756',
  '17.323522915498703,-88.3832775663601',
  '4.246782213565523,66.48852816008434',
  '3.6680901970686763,-63.635006558579875',
  '19.398197043239886,-95.88310114083951',
  '10.495128632644757,-39.15155140809246',
  '5.824582803960839,-13.610996271576852',
  '3.1957112422714085,-81.71669017709647',
  '10.433344851231102,-79.72837617300237',
  '9.275597238846595,-98.8712926162705',
  '12.843733742755944,33.50714506627196',
  '4.304918715148016,-93.50598115024678',
  '19.148309730364033,-95.96081184529413',
  '16.48737869615314,-11.639564423352283',
  '19.913278043351237,99.8344133332296',
  '19.39627719206612,99.45143746799172',
  '0.11797159900006626,-73.5927216207487'],
 'historical_values': [0.5,
  0.2,
  -14.639878836228101,
  2.456355072354658,
  6.56

In [33]:
pwd

'/home/ec2-user/SageMaker/projects'

In [31]:
algo.searcher.test_optimizer.params_history

[{'height': 46.39878836228101, 'width': 11.973169683940732},
 {'height': -25.091976230527507, 'width': 19.01428612819832},
 {'height': -68.79627191151269, 'width': 3.119890406724053},
 {'height': 20.223002348641756, 'width': 14.161451555920909},
 {'height': -88.3832775663601, 'width': 17.323522915498703},
 {'height': 66.48852816008434, 'width': 4.246782213565523},
 {'height': -63.635006558579875, 'width': 3.6680901970686763},
 {'height': -95.88310114083951, 'width': 19.398197043239886},
 {'height': -39.15155140809246, 'width': 10.495128632644757},
 {'height': -13.610996271576852, 'width': 5.824582803960839},
 {'height': -81.71669017709647, 'width': 3.1957112422714085},
 {'height': -79.72837617300237, 'width': 10.433344851231102},
 {'height': -98.8712926162705, 'width': 9.275597238846595},
 {'height': 33.50714506627196, 'width': 12.843733742755944},
 {'height': -95.96081184529413, 'width': 19.148309730364033},
 {'height': -93.50598115024678, 'width': 4.304918715148016},
 {'height': -11.